# 0. Import Dependencies

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import tensorflow as tf
import math

from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, classification_report
from tensorflow.keras.utils import to_categorical

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.layers import (LSTM, Dense, Concatenate, Attention, Dropout, Softmax,
                                     Input, Flatten, Activation, Bidirectional, Permute, multiply, 
                                     ConvLSTM2D, MaxPooling3D, TimeDistributed, Conv2D, MaxPooling2D)

from scipy import stats

os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"
tf.get_logger().setLevel("ERROR")
tf.autograph.set_verbosity(1)

import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

# 1. Keypoints using MP Pose

In [2]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False                 
    results = model.process(image)                
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )

In [5]:
cap = cv2.VideoCapture(0)
HEIGHT = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
WIDTH = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
FPS = int(cap.get(cv2.CAP_PROP_FPS))

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, pose)
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass

        draw_landmarks(image, results)               
        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# 2. Extract Keypoints

In [6]:
pose = []
if results.pose_landmarks is not None:
    for res in results.pose_landmarks.landmark:
        test = np.array([res.x, res.y, res.z, res.visibility])
        pose.append(test)
else:
    print("No pose landmarks found.")


In [7]:
num_landmarks = len(landmarks)
num_values = len(test)
num_input_values = num_landmarks*num_values

In [8]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)

In [9]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return pose

# 3. Setup Folders for Collection

In [10]:
DATA_PATH = os.path.join(os. getcwd(),'data') 
print(DATA_PATH)
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)
actions = np.array(['curl', 'press', 'squat'])
num_classes = len(actions)
no_sequences = 0
sequence_length = FPS*1
start_folder = 161

C:\Users\nithinthangallapelly\Desktop\project\data


In [11]:
for action in actions:     
    for sequence in range(start_folder,no_sequences+start_folder):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))  
        except:
            pass

# 4. Collect Keypoint Values for Training and Testing

In [12]:
colors = [(245,117,16), (117,245,16), (16,117,245)]

In [13]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    for idx, action in enumerate(actions):
        for sequence in range(start_folder, start_folder+no_sequences):
            for frame_num in range(sequence_length):
                ret, frame = cap.read()
                image, results = mediapipe_detection(frame, pose)
                try:
                    landmarks = results.pose_landmarks.landmark
                except:
                    pass
            
                draw_landmarks(image, results) 
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    
                    cv2.putText(image, 'Collecting {} Video # {}'.format(action, sequence), (15,30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 8, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting {} Video # {}'.format(action, sequence), (15,30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, colors[idx], 4, cv2.LINE_AA)
                    
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(1000)
                else: 
                    cv2.putText(image, 'Collecting {} Video # {}'.format(action, sequence), (15,30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 8, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting {} Video # {}'.format(action, sequence), (15,30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, colors[idx], 4, cv2.LINE_AA)
                
                    cv2.imshow('OpenCV Feed', image)
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [14]:
cap.release()
cv2.destroyAllWindows()

# 5. Preprocess Data and Create Labels/Features

In [15]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'curl': 0, 'press': 1, 'squat': 2}

In [16]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):         
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)  
            
        sequences.append(window)
        labels.append(label_map[action])

In [17]:
np.array(labels).shape


(180,)

In [18]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)
print(X.shape, y.shape)

(180, 30, 132) (180, 3)


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1)
print(X_train.shape, y_train.shape)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=2)

(162, 30, 132) (162, 3)


# 6. Build and Train Neural Networks

In [20]:
es_callback = EarlyStopping(monitor='val_loss', min_delta=5e-4, patience=10, verbose=0, mode='min')
lr_callback = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001, verbose=0, mode='min')
chkpt_callback = ModelCheckpoint(filepath=DATA_PATH, monitor='val_loss', verbose=0, save_best_only=True, 
                                 save_weights_only=False, mode='min', save_freq=1)

opt = tf.keras.optimizers.Adam(learning_rate=0.01)
batch_size = 32
max_epochs = 200

## 6a. LSTM

In [21]:
NAME = f"ExerciseRecognition-LSTM-{int(time.time())}"
log_dir = os.path.join(os.getcwd(), 'logs', NAME,'')
tb_callback = TensorBoard(log_dir=log_dir)

callbacks = [tb_callback, es_callback, lr_callback, chkpt_callback]

In [22]:
lstm = Sequential()
lstm.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(sequence_length, num_input_values)))
lstm.add(LSTM(256, return_sequences=True, activation='relu'))
lstm.add(LSTM(128, return_sequences=False, activation='relu'))
lstm.add(Dense(128, activation='relu'))
lstm.add(Dense(64, activation='relu'))
lstm.add(Dense(actions.shape[0], activation='softmax'))
print(lstm.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 128)           133632    
                                                                 
 lstm_1 (LSTM)               (None, 30, 256)           394240    
                                                                 
 lstm_2 (LSTM)               (None, 128)               197120    
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 3)                 195       
                                                                 
Total params: 749,955
Trainable params: 749,955
Non-trai

In [23]:
lstm.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
lstm.fit(X_train, y_train, batch_size=batch_size, epochs=max_epochs, validation_data=(X_val, y_val), callbacks=callbacks)

Epoch 1/200
5/5 [==============================] - 7s 353ms/step - loss: 1034446.7500 - categorical_accuracy: 0.3101 - val_loss: 610.8690 - val_categorical_accuracy: 0.3030 - lr: 0.0100
Epoch 2/200
5/5 [==============================] - 1s 170ms/step - loss: 1679.5549 - categorical_accuracy: 0.3411 - val_loss: 1014.2176 - val_categorical_accuracy: 0.3030 - lr: 0.0100
Epoch 3/200
5/5 [==============================] - 1s 169ms/step - loss: 1600.0239 - categorical_accuracy: 0.3411 - val_loss: 261.2590 - val_categorical_accuracy: 0.3030 - lr: 0.0100
Epoch 4/200
5/5 [==============================] - 1s 168ms/step - loss: 316.3133 - categorical_accuracy: 0.3488 - val_loss: 644.4307 - val_categorical_accuracy: 0.3030 - lr: 0.0100
Epoch 5/200
5/5 [==============================] - 1s 165ms/step - loss: 500.5238 - categorical_accuracy: 0.3333 - val_loss: 228.1315 - val_categorical_accuracy: 0.3939 - lr: 0.0100
Epoch 6/200
5/5 [==============================] - 1s 167ms/step - loss: 497.6654 -

Epoch 46/200
5/5 [==============================] - 1s 194ms/step - loss: 1.1327 - categorical_accuracy: 0.3178 - val_loss: 1.0994 - val_categorical_accuracy: 0.3939 - lr: 8.0000e-05
Epoch 47/200
5/5 [==============================] - 1s 205ms/step - loss: 1.1133 - categorical_accuracy: 0.3178 - val_loss: 1.0989 - val_categorical_accuracy: 0.3939 - lr: 8.0000e-05


## 6b. LSTM + Attention

In [24]:
NAME = f"ExerciseRecognition-AttnLSTM-{int(time.time())}"
log_dir = os.path.join(os.getcwd(), 'logs', NAME,'')
tb_callback = TensorBoard(log_dir=log_dir)

callbacks = [tb_callback, es_callback, lr_callback, chkpt_callback]

In [25]:
def attention_block(inputs, time_steps):
    a = Permute((2, 1))(inputs)
    a = Dense(time_steps, activation='softmax')(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = multiply([inputs, a_probs], name='attention_mul') 
    return output_attention_mul

In [26]:
HIDDEN_UNITS = 256
inputs = Input(shape=(sequence_length, num_input_values))
lstm_out = Bidirectional(LSTM(HIDDEN_UNITS, return_sequences=True))(inputs)
attention_mul = attention_block(lstm_out, sequence_length)
attention_mul = Flatten()(attention_mul)
x = Dense(2*HIDDEN_UNITS, activation='relu')(attention_mul)
x = Dropout(0.5)(x)
x = Dense(actions.shape[0], activation='softmax')(x)
AttnLSTM = Model(inputs=[inputs], outputs=x)
print(AttnLSTM.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 30, 132)]    0           []                               
                                                                                                  
 bidirectional (Bidirectional)  (None, 30, 512)      796672      ['input_1[0][0]']                
                                                                                                  
 permute (Permute)              (None, 512, 30)      0           ['bidirectional[0][0]']          
                                                                                                  
 dense_3 (Dense)                (None, 512, 30)      930         ['permute[0][0]']                
                                                                                              

In [27]:
AttnLSTM.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
AttnLSTM.fit(X_train, y_train, batch_size=batch_size, epochs=max_epochs, validation_data=(X_val, y_val), callbacks=callbacks)

Epoch 1/200
5/5 [==============================] - 8s 549ms/step - loss: 1.1004 - categorical_accuracy: 0.3566 - val_loss: 1.0999 - val_categorical_accuracy: 0.3030 - lr: 1.6000e-05
Epoch 2/200
5/5 [==============================] - 1s 258ms/step - loss: 1.0951 - categorical_accuracy: 0.3721 - val_loss: 1.1003 - val_categorical_accuracy: 0.3030 - lr: 1.6000e-05
Epoch 3/200
5/5 [==============================] - 1s 259ms/step - loss: 1.0952 - categorical_accuracy: 0.4031 - val_loss: 1.0956 - val_categorical_accuracy: 0.3030 - lr: 1.6000e-05
Epoch 4/200
5/5 [==============================] - 1s 259ms/step - loss: 1.0953 - categorical_accuracy: 0.3643 - val_loss: 1.0917 - val_categorical_accuracy: 0.6061 - lr: 1.6000e-05
Epoch 5/200
5/5 [==============================] - 1s 258ms/step - loss: 1.0908 - categorical_accuracy: 0.4574 - val_loss: 1.0905 - val_categorical_accuracy: 0.5152 - lr: 1.6000e-05
Epoch 6/200
5/5 [==============================] - 1s 254ms/step - loss: 1.0866 - categori

Epoch 46/200
5/5 [==============================] - 1s 252ms/step - loss: 0.5563 - categorical_accuracy: 0.8837 - val_loss: 0.6216 - val_categorical_accuracy: 0.7879 - lr: 1.6000e-05
Epoch 47/200
5/5 [==============================] - 1s 253ms/step - loss: 0.5357 - categorical_accuracy: 0.8837 - val_loss: 0.6129 - val_categorical_accuracy: 0.8485 - lr: 1.6000e-05
Epoch 48/200
5/5 [==============================] - 1s 255ms/step - loss: 0.5445 - categorical_accuracy: 0.9070 - val_loss: 0.6027 - val_categorical_accuracy: 0.8182 - lr: 1.6000e-05
Epoch 49/200
5/5 [==============================] - 1s 255ms/step - loss: 0.5454 - categorical_accuracy: 0.9070 - val_loss: 0.5864 - val_categorical_accuracy: 0.8182 - lr: 1.6000e-05
Epoch 50/200
5/5 [==============================] - 1s 256ms/step - loss: 0.5062 - categorical_accuracy: 0.9070 - val_loss: 0.5643 - val_categorical_accuracy: 0.8788 - lr: 1.6000e-05
Epoch 51/200
5/5 [==============================] - 1s 261ms/step - loss: 0.5085 - ca

Epoch 91/200
5/5 [==============================] - 2s 404ms/step - loss: 0.2325 - categorical_accuracy: 0.9612 - val_loss: 0.3367 - val_categorical_accuracy: 0.8485 - lr: 1.6000e-05
Epoch 92/200
5/5 [==============================] - 2s 354ms/step - loss: 0.2323 - categorical_accuracy: 0.9612 - val_loss: 0.3601 - val_categorical_accuracy: 0.8788 - lr: 1.6000e-05
Epoch 93/200
5/5 [==============================] - 2s 314ms/step - loss: 0.2298 - categorical_accuracy: 0.9690 - val_loss: 0.3670 - val_categorical_accuracy: 0.8788 - lr: 1.6000e-05
Epoch 94/200
5/5 [==============================] - 2s 348ms/step - loss: 0.2332 - categorical_accuracy: 0.9612 - val_loss: 0.3453 - val_categorical_accuracy: 0.8485 - lr: 1.6000e-05
Epoch 95/200
5/5 [==============================] - 1s 288ms/step - loss: 0.2244 - categorical_accuracy: 0.9612 - val_loss: 0.3176 - val_categorical_accuracy: 0.8788 - lr: 1.6000e-05
Epoch 96/200
5/5 [==============================] - 1s 271ms/step - loss: 0.2240 - ca

In [28]:
models = {
    'LSTM': lstm, 
    'LSTM_Attention_128HUs': AttnLSTM, 
}

# 7a. Save Weights

In [29]:
for model_name, model in models.items():
    save_dir = os.path.join(os.getcwd(), f"{model_name}.h5")
    model.save(save_dir)

# 7b. Load Weights

In [30]:
for model_name, model in models.items():
    load_dir = os.path.join(os.getcwd(), f"{model_name}.h5")
    model.load_weights(load_dir)

# 8. Make Predictions

In [31]:
for model in models.values():
    res = model.predict(X_test, verbose=0)   

# 9. Evaluations using Confusion Matrix and Accuracy

In [32]:
eval_results = {}
eval_results['confusion matrix'] = None
eval_results['accuracy'] = None
eval_results['precision'] = None
eval_results['recall'] = None
eval_results['f1 score'] = None

confusion_matrices = {}
classification_accuracies = {}   
precisions = {}
recalls = {}
f1_scores = {} 

## 9a. Confusion Matrices

In [33]:
for model_name, model in models.items():
    yhat = model.predict(X_test, verbose=0)
    ytrue = np.argmax(y_test, axis=1).tolist()
    yhat = np.argmax(yhat, axis=1).tolist()
    confusion_matrices[model_name] = multilabel_confusion_matrix(ytrue, yhat)
    print(f"{model_name} confusion matrix: {os.linesep}{confusion_matrices[model_name]}")
eval_results['confusion matrix'] = confusion_matrices

LSTM confusion matrix: 
[[[ 0 12]
  [ 0  6]]

 [[10  0]
  [ 8  0]]

 [[14  0]
  [ 4  0]]]
LSTM_Attention_128HUs confusion matrix: 
[[[12  0]
  [ 1  5]]

 [[10  0]
  [ 0  8]]

 [[13  1]
  [ 0  4]]]


## 9b. Accuracy

In [34]:
for model_name, model in models.items():
    yhat = model.predict(X_test, verbose=0)
    ytrue = np.argmax(y_test, axis=1).tolist()
    yhat = np.argmax(yhat, axis=1).tolist()
    classification_accuracies[model_name] = accuracy_score(ytrue, yhat)    
    print(f"{model_name} classification accuracy = {round(classification_accuracies[model_name]*100,3)}%")
eval_results['accuracy'] = classification_accuracies

LSTM classification accuracy = 33.333%
LSTM_Attention_128HUs classification accuracy = 94.444%


## 9c. Precision, Recall, and F1 Score

In [35]:
for model_name, model in models.items():
    yhat = model.predict(X_test, verbose=0)
    ytrue = np.argmax(y_test, axis=1).tolist()
    yhat = np.argmax(yhat, axis=1).tolist()
    report = classification_report(ytrue, yhat, target_names=actions, output_dict=True,zero_division=1)
    
    precisions[model_name] = report['weighted avg']['precision']
    recalls[model_name] = report['weighted avg']['recall']
    f1_scores[model_name] = report['weighted avg']['f1-score'] 
   
    print(f"{model_name} weighted average precision = {round(precisions[model_name],3)}")
    print(f"{model_name} weighted average recall = {round(recalls[model_name],3)}")
    print(f"{model_name} weighted average f1-score = {round(f1_scores[model_name],3)}\n")

 
eval_results['precision'] = precisions
eval_results['recall'] = recalls
eval_results['f1 score'] = f1_scores

LSTM weighted average precision = 0.778
LSTM weighted average recall = 0.333
LSTM weighted average f1-score = 0.167

LSTM_Attention_128HUs weighted average precision = 0.956
LSTM_Attention_128HUs weighted average recall = 0.944
LSTM_Attention_128HUs weighted average f1-score = 0.945



# 10. Choose Model to Test in Real Time

In [36]:
model = AttnLSTM
model_name = 'AttnLSTM'

# 11. Calculate Joint Angles & Count Reps

In [37]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [38]:
def get_coordinates(landmarks, mp_pose, side, joint):
    coord = getattr(mp_pose.PoseLandmark,side.upper()+"_"+joint.upper())
    x_coord_val = landmarks[coord.value].x
    y_coord_val = landmarks[coord.value].y
    return [x_coord_val, y_coord_val]            

In [39]:
def viz_joint_angle(image, angle, joint):
    cv2.putText(image, str(int(angle)), 
                   tuple(np.multiply(joint, [640, 480]).astype(int)), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                        )
    return

In [40]:
def count_reps(image, current_action, landmarks, mp_pose):
    global curl_counter, press_counter, squat_counter, curl_stage, press_stage, squat_stage
    
    if current_action == 'curl':
        shoulder = get_coordinates(landmarks, mp_pose, 'left', 'shoulder')
        elbow = get_coordinates(landmarks, mp_pose, 'left', 'elbow')
        wrist = get_coordinates(landmarks, mp_pose, 'left', 'wrist')
        
        angle = calculate_angle(shoulder, elbow, wrist)
        # curl counter logic
        if angle < 30:
            curl_stage = "up" 
        if angle > 160 and curl_stage =='up':
            curl_stage="down"  
            curl_counter +=1
        press_stage = None
        squat_stage = None
        viz_joint_angle(image, angle, elbow)
        # press counter Logic
    elif current_action == 'press':
        shoulder = get_coordinates(landmarks, mp_pose, 'left', 'shoulder')
        elbow = get_coordinates(landmarks, mp_pose, 'left', 'elbow')
        wrist = get_coordinates(landmarks, mp_pose, 'left', 'wrist')
        elbow_angle = calculate_angle(shoulder, elbow, wrist)
        shoulder2elbow_dist = abs(math.dist(shoulder,elbow))
        shoulder2wrist_dist = abs(math.dist(shoulder,wrist))
        if (elbow_angle > 130) and (shoulder2elbow_dist < shoulder2wrist_dist):
            press_stage = "up"
        if (elbow_angle < 50) and (shoulder2elbow_dist > shoulder2wrist_dist) and (press_stage =='up'):
            press_stage='down'
            press_counter += 1
        curl_stage = None
        squat_stage = None
        viz_joint_angle(image, elbow_angle, elbow)
        # Squat counter logic
    elif current_action == 'squat':
        left_shoulder = get_coordinates(landmarks, mp_pose, 'left', 'shoulder')
        left_hip = get_coordinates(landmarks, mp_pose, 'left', 'hip')
        left_knee = get_coordinates(landmarks, mp_pose, 'left', 'knee')
        left_ankle = get_coordinates(landmarks, mp_pose, 'left', 'ankle')
        right_shoulder = get_coordinates(landmarks, mp_pose, 'right', 'shoulder')
        right_hip = get_coordinates(landmarks, mp_pose, 'right', 'hip')
        right_knee = get_coordinates(landmarks, mp_pose, 'right', 'knee')
        right_ankle = get_coordinates(landmarks, mp_pose, 'right', 'ankle')
        
        left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
        
        left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
        right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
        
        # Squat counter logic
        thr = 165
        if (left_knee_angle < thr) and (right_knee_angle < thr) and (left_hip_angle < thr) and (right_hip_angle < thr):
            squat_stage = "down"
        if (left_knee_angle > thr) and (right_knee_angle > thr) and (left_hip_angle > thr) and (right_hip_angle > thr) and (squat_stage =='down'):
            squat_stage='up'
            squat_counter += 1
        curl_stage = None
        press_stage = None
        viz_joint_angle(image, left_knee_angle, left_knee)
        viz_joint_angle(image, left_hip_angle, left_hip)
        
    else:
        pass

# 12. Test in Real Time

In [41]:
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):        
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [46]:
sequence = []
predictions = []
res = []
threshold = 0.5 
current_action = ''

curl_counter = 0
press_counter = 0
squat_counter = 0
curl_stage = None
press_stage = None
squat_stage = None

cap = cv2.VideoCapture(0)

fourcc = cv2.VideoWriter_fourcc('M','J','P','G') 
HEIGHT = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) 
WIDTH = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
FPS = int(cap.get(cv2.CAP_PROP_FPS))

video_name = os.path.join(os.getcwd(),f"{model_name}_real_time_test2.avi")
out = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*"MJPG"), FPS, (WIDTH,HEIGHT))

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, pose)
        draw_landmarks(image, results)
        
        keypoints = extract_keypoints(results)        
        sequence.append(keypoints)      
        sequence = sequence[-sequence_length:]
              
        if len(sequence) == sequence_length:
            res = model.predict(np.expand_dims(sequence, axis=0), verbose=0)[0]           
            predictions.append(np.argmax(res))
            current_action = actions[np.argmax(res)]
            confidence = np.max(res)
            
            if confidence < threshold:
                current_action = ''
            image = prob_viz(res, actions, image, colors)
            
            try:
                landmarks = results.pose_landmarks.landmark
                count_reps(
                    image, current_action, landmarks, mp_pose)
            except:
                pass
            cv2.rectangle(image, (0,0), (640, 40), colors[np.argmax(res)], -1)
            cv2.putText(image, 'curl ' + str(curl_counter), (3,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, 'press ' + str(press_counter), (240,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, 'squat ' + str(squat_counter), (490,30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
         
        cv2.imshow('OpenCV Feed', image)
        if ret == True:
            out.write(image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [43]:
cap.release()
out.release()
cv2.destroyAllWindows()